In [ ]:
import customtkinter as ctk
import threading
import requests
import json
import re
import random
import math
import os
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor
from collections import Counter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# --- BACKEND LOGIC (The Meal Finding Engine) ---
class MealFinder:
    def __init__(self):
        self.url = "https://api.hfs.purdue.edu/menus/v3/GraphQL"
        self.headers = {"Content-Type": "application/json"}
        # Updated query to pull traits for filtering
        self.query = """
        query GetMenu($courtName: String!, $date: Date!) {
          diningCourtByName(name: $courtName) {
            name
            dailyMenu(date: $date) {
              meals { name, stations { name, items { displayName, item { traits { name }, nutritionFacts { name, label } } } } }
            }
          }
        }
        """
        self.dining_courts = ["Wiley", "Earhart", "Windsor", "Ford", "Hillenbrand"]
        self.todays_date = datetime.now().strftime('%Y-%m-%d')
        self.cache_file = f"menu_cache_{self.todays_date}.json"
        self.master_item_list = []
        self.data_loaded = False

    def _get_numeric_value(self, label_str):
        if not label_str: return 0.0
        numeric_part = re.search(r'[\d.]+', label_str)
        return float(numeric_part.group(0)) if numeric_part else 0.0

    def _calculate_score(self, meal_plan, targets, weights, penalties):
        if not meal_plan: return float('inf'), {}
        totals = {'p': sum(item['p'] for item in meal_plan), 'c': sum(item['c'] for item in meal_plan), 'f': sum(item['f'] for item in meal_plan)}
        errors = {'p': totals['p'] - targets['p'], 'c': totals['c'] - targets['c'], 'f': totals['f'] - targets['f']}
        if errors['p'] < 0: errors['p'] *= penalties['under_p']
        if errors['c'] > 0: errors['c'] *= penalties['over_c']
        if errors['f'] > 0: errors['f'] *= penalties['over_f']
        score = (weights['p'] * (errors['p']**2) + weights['c'] * (errors['c']**2) + weights['f'] * (errors['f']**2))**0.5
        return score, totals

    def _get_menu_data_for_court(self, court, cached_data):
        menu_data = cached_data.get(court)
        if menu_data: return court, menu_data, False
        try:
            variables = {"courtName": court, "date": self.todays_date}
            resp = requests.post(self.url, json={"query": self.query, "variables": variables}, headers=self.headers)
            resp.raise_for_status()
            return court, resp.json(), True
        except requests.exceptions.RequestException:
            return court, None, False

    def find_best_meal(self, targets, meal_periods_to_check, exclusion_list=[], dietary_filters={}):
        if not self.data_loaded:
            # ... (Caching and data loading logic is the same) ...
            cached_data, needs_to_save_cache = {}, False
            if os.path.exists(self.cache_file):
                with open(self.cache_file, 'r') as f:
                    try: cached_data = json.load(f).get("data", {})
                    except: pass
            
            with ThreadPoolExecutor() as executor:
                future_to_court = {executor.submit(self._get_menu_data_for_court, court, cached_data): court for court in self.dining_courts}
                for future in future_to_court:
                    court, menu_data, was_fetched = future.result()
                    if was_fetched:
                        cached_data[court], needs_to_save_cache = menu_data, True
                    
                    if menu_data and 'data' in menu_data and menu_data.get('data', {}).get('diningCourtByName'):
                        for meal in menu_data['data']['diningCourtByName']['dailyMenu']['meals']:
                            for station in meal['stations']:
                                for item_appearance in station['items']:
                                    core_item = item_appearance.get('item')
                                    if core_item and core_item.get('nutritionFacts'):
                                        macros = {'Protein': 0, 'Total Carbohydrate': 0, 'Total fat': 0}
                                        for fact in core_item['nutritionFacts']:
                                            if fact['name'] in macros: macros[fact['name']] = self._get_numeric_value(fact.get('label'))
                                        
                                        traits = [trait['name'] for trait in core_item.get('traits', []) if trait] if core_item.get('traits') else []
                                        if sum(macros.values()) > 0:
                                            self.master_item_list.append({
                                                "name": item_appearance['displayName'],
                                                "p": macros['Protein'], "c": macros['Total Carbohydrate'], "f": macros['Total fat'],
                                                "court": court, "meal_name": meal['name'], "traits": traits
                                            })
            if needs_to_save_cache:
                with open(self.cache_file, 'w') as f: json.dump({"timestamp": datetime.now().isoformat(), "data": cached_data}, f)
            self.data_loaded = True
        
        # ** NEW: Apply dietary filters **
        filtered_master_list = []
        for item in self.master_item_list:
            traits = item.get('traits', [])
            passes_filter = True
            # Check positive requirements (like Vegetarian)
            if dietary_filters.get("Vegetarian") and "Vegetarian" not in traits: passes_filter = False
            if dietary_filters.get("Vegan") and "Vegan" not in traits: passes_filter = False
            # Check negative requirements (allergens)
            if dietary_filters.get("No Gluten") and "Gluten" in traits: passes_filter = False
            if dietary_filters.get("No Nuts") and ("Tree Nuts" in traits or "Peanuts" in traits): passes_filter = False
            if dietary_filters.get("No Eggs") and "Eggs" in traits: passes_filter = False

            if passes_filter:
                filtered_master_list.append(item)
        
        available_items = [item for item in filtered_master_list if item['name'] not in exclusion_list and item['meal_name'] in meal_periods_to_check]
        if len(available_items) < 2: return None

        # ... (Simulated Annealing logic is the same) ...
        best_solution, best_score, best_totals = None, float('inf'), {}
        weights = {'p': 3.0, 'c': 1.0, 'f': 1.5}
        penalties = {'under_p': 2, 'over_c': 1.2, 'over_f': 3}
        temp, cooling_rate, iterations = 10000, 0.99, 3000
        current_solution = random.sample(available_items, min(4, len(available_items)))
        for _ in range(iterations):
            if temp <= 1: break
            neighbor = list(current_solution)
            if len(neighbor) > 1 and random.random() < 0.7: neighbor[random.randrange(len(neighbor))] = random.choice(available_items)
            elif len(neighbor) < 5 and random.random() < 0.5:
                if len(available_items) > len(neighbor): neighbor.append(random.choice([i for i in available_items if i not in neighbor]))
            elif len(neighbor) > 2: neighbor.pop(random.randrange(len(neighbor)))
            current_score, _ = self._calculate_score(current_solution, targets, weights, penalties)
            neighbor_score, neighbor_totals = self._calculate_score(neighbor, targets, weights, penalties)
            if neighbor_score < current_score or random.random() < math.exp((current_score - neighbor_score) / temp): current_solution = neighbor
            if neighbor_score < best_score: best_score, best_totals, best_solution = neighbor_score, neighbor_totals, neighbor
            temp *= cooling_rate
        
        if not best_solution: return None
        return {"score": best_score, "court": best_solution[0]['court'], "meal_name": best_solution[0]['meal_name'], "plan": best_solution, "totals": best_totals}

# --- FRONTEND (GUI Application) ---
class App(ctk.CTk):
    def __init__(self, meal_finder):
        super().__init__()
        self.meal_finder = meal_finder
        self.exclusion_list = []
        self.profiles = {}
        self.profiles_file = "macro_profiles.json"
        
        self.title("Purdue Automated Meal Planner")
        self.geometry("1000x700")
        ctk.set_appearance_mode("dark")
        ctk.set_default_color_theme("blue")
        self.grid_columnconfigure(1, weight=1)
        self.grid_rowconfigure(0, weight=1)
        
        # --- Left Frame (Inputs) ---
        self.left_frame = ctk.CTkFrame(self, width=280, corner_radius=0)
        self.left_frame.grid(row=0, column=0, sticky="nsw")
        
        # ... (Input widgets)
        self.title_label = ctk.CTkLabel(self.left_frame, text="Meal Planner", font=ctk.CTkFont(size=20, weight="bold"))
        self.title_label.grid(row=0, column=0, columnspan=2, padx=20, pady=(20, 10))
        self.meal_period_menu = ctk.CTkOptionMenu(self.left_frame, values=["Breakfast", "Lunch", "Dinner"])
        self.meal_period_menu.grid(row=1, column=0, columnspan=2, padx=20, pady=10)
        self.protein_entry = ctk.CTkEntry(self.left_frame, placeholder_text="Target Protein (g)")
        self.protein_entry.grid(row=2, column=0, columnspan=2, padx=20, pady=10)
        self.carb_entry = ctk.CTkEntry(self.left_frame, placeholder_text="Target Carbs (g)")
        self.carb_entry.grid(row=3, column=0, columnspan=2, padx=20, pady=10)
        self.fat_entry = ctk.CTkEntry(self.left_frame, placeholder_text="Target Fat (g)")
        self.fat_entry.grid(row=4, column=0, columnspan=2, padx=20, pady=10)

        # ** NEW: Macro Profiles **
        self.profile_label = ctk.CTkLabel(self.left_frame, text="Macro Profiles")
        self.profile_label.grid(row=5, column=0, columnspan=2, padx=20, pady=(10,0))
        self.profile_var = ctk.StringVar(value="Select a Profile")
        self.profile_menu = ctk.CTkOptionMenu(self.left_frame, variable=self.profile_var, values=["Loading..."])
        self.profile_menu.grid(row=6, column=0, padx=(20,5), pady=5, sticky="ew")
        self.load_profile_button = ctk.CTkButton(self.left_frame, text="Load", width=60, command=self.load_selected_profile)
        self.load_profile_button.grid(row=6, column=1, padx=(5,20), pady=5)
        self.profile_name_entry = ctk.CTkEntry(self.left_frame, placeholder_text="New Profile Name")
        self.profile_name_entry.grid(row=7, column=0, padx=(20,5), pady=5, sticky="ew")
        self.save_profile_button = ctk.CTkButton(self.left_frame, text="Save", width=60, command=self.save_current_profile)
        self.save_profile_button.grid(row=7, column=1, padx=(5,20), pady=5)

        # ** NEW: Dietary Filters **
        self.filter_label = ctk.CTkLabel(self.left_frame, text="Dietary Filters")
        self.filter_label.grid(row=8, column=0, columnspan=2, padx=20, pady=(10,0))
        self.filters = {}
        filter_options = ["Vegetarian", "Vegan", "No Gluten", "No Nuts", "No Eggs"]
        for i, option in enumerate(filter_options):
            self.filters[option] = ctk.BooleanVar()
            chk = ctk.CTkCheckBox(self.left_frame, text=option, variable=self.filters[option])
            chk.grid(row=9+i, column=0, columnspan=2, padx=20, pady=5, sticky="w")
        
        # ... (Buttons)
        self.find_button = ctk.CTkButton(self.left_frame, text="Find Meal", command=self.start_meal_search)
        self.find_button.grid(row=14, column=0, columnspan=2, padx=20, pady=10)
        self.reset_button = ctk.CTkButton(self.left_frame, text="Reset Exclusions", command=self.reset_exclusions, fg_color="transparent", border_width=2)
        self.reset_button.grid(row=15, column=0, columnspan=2, padx=20, pady=(10, 20))

        # --- Right Frame (Outputs) ---
        self.right_frame = ctk.CTkFrame(self, corner_radius=0, fg_color="transparent")
        self.right_frame.grid(row=0, column=1, sticky="nsew", padx=20, pady=20)
        # ... (Right frame configuration)
        self.right_frame.grid_rowconfigure(1, weight=1)
        self.right_frame.grid_columnconfigure(0, weight=1)
        self.status_label = ctk.CTkLabel(self.right_frame, text="Enter your macros and find a meal!", font=ctk.CTkFont(size=16))
        self.status_label.grid(row=0, column=0, pady=10, sticky="ew")
        self.result_frame = ctk.CTkScrollableFrame(self.right_frame, label_text="Recommendation")
        self.result_frame.grid(row=1, column=0, sticky="nsew", pady=(0,10))

        # ** NEW: Chart Frame **
        self.chart_frame = ctk.CTkFrame(self.right_frame, height=200)
        self.chart_frame.grid(row=2, column=0, sticky="nsew")
        self.canvas = None

        self.load_profiles_from_file() # Load profiles on startup

    # ** NEW: Profile Management Functions **
    def load_profiles_from_file(self):
        if os.path.exists(self.profiles_file):
            with open(self.profiles_file, 'r') as f:
                self.profiles = json.load(f)
        self.update_profile_menu()

    def update_profile_menu(self):
        profile_names = list(self.profiles.keys())
        self.profile_menu.configure(values=profile_names if profile_names else ["No profiles saved"])

    def load_selected_profile(self):
        profile_name = self.profile_var.get()
        if profile_name in self.profiles:
            macros = self.profiles[profile_name]
            self.protein_entry.delete(0, "end"); self.protein_entry.insert(0, macros.get('p', ''))
            self.carb_entry.delete(0, "end"); self.carb_entry.insert(0, macros.get('c', ''))
            self.fat_entry.delete(0, "end"); self.fat_entry.insert(0, macros.get('f', ''))

    def save_current_profile(self):
        profile_name = self.profile_name_entry.get()
        if profile_name:
            try:
                self.profiles[profile_name] = {
                    'p': self.protein_entry.get(),
                    'c': self.carb_entry.get(),
                    'f': self.fat_entry.get()
                }
                with open(self.profiles_file, 'w') as f:
                    json.dump(self.profiles, f, indent=4)
                self.update_profile_menu()
                self.profile_name_entry.delete(0, "end")
            except Exception as e:
                print(f"Error saving profile: {e}")

    # ... (start_meal_search is the same)
    def start_meal_search(self):
        threading.Thread(target=self.find_meal_logic, daemon=True).start()

    def find_meal_logic(self):
        self.find_button.configure(state="disabled", text="Searching...")
        self.status_label.configure(text="Analyzing menus...")
        try:
            targets = {'p': int(self.protein_entry.get() or 0), 'c': int(self.carb_entry.get() or 0), 'f': int(self.fat_entry.get() or 0)}
            meal_choice = self.meal_period_menu.get()
            meal_periods = ["Lunch", "Late Lunch"] if meal_choice == "Lunch" else [meal_choice]
            # Get selected filters
            active_filters = {key: var.get() for key, var in self.filters.items() if var.get()}
            result = self.meal_finder.find_best_meal(targets, meal_periods, self.exclusion_list, active_filters)
            self.after(0, self.display_result, result, targets)
        except ValueError:
            self.status_label.configure(text="Error: Please enter valid numbers for macros.")
        finally:
            self.find_button.configure(state="normal", text="Find Meal")

    # ** MODIFIED: display_result now creates chart **
    def display_result(self, result, targets):
        for widget in self.result_frame.winfo_children(): widget.destroy()
        if not result or not result.get("plan"):
            self.status_label.configure(text="Could not find a suitable meal.")
            return

        b = result
        self.status_label.configure(text=f"Best match is for {b['meal_name']} at {b['court']}")
        
        # ... (display recommendation list)
        ctk.CTkLabel(self.result_frame, text="--- Items to Get ---", font=ctk.CTkFont(weight="bold")).pack(pady=(10,5))
        for item in b['plan']:
            item_frame = ctk.CTkFrame(self.result_frame)
            item_frame.pack(fill="x", padx=10, pady=2)
            ctk.CTkLabel(item_frame, text=f"{item['name']} (P:{item['p']:.0f}g, C:{item['c']:.0f}g, F:{item['f']:.0f}g)").pack(side="left", padx=10)
            ctk.CTkButton(item_frame, text="Remove", width=60, fg_color="red", command=lambda name=item['name']: self.remove_and_recalibrate(name)).pack(side="right", padx=10)

        # Update chart
        self.update_macro_chart(b['totals'])

    # ** NEW: Charting Function **
    def update_macro_chart(self, totals):
        if self.canvas:
            self.canvas.get_tk_widget().destroy()

        p, c, f = totals.get('p', 0), totals.get('c', 0), totals.get('f', 0)
        
        # Avoid creating an empty chart
        if p + c + f == 0: return

        calories_from_p = p * 4
        calories_from_c = c * 4
        calories_from_f = f * 9
        
        sizes = [calories_from_p, calories_from_c, calories_from_f]
        labels = [f'Protein\n{p:.0f}g', f'Carbs\n{c:.0f}g', f'Fat\n{f:.0f}g']
        colors = ['#3498db', '#2ecc71', '#e74c3c'] # Blue, Green, Red

        fig, ax = plt.subplots(figsize=(4, 2.5), dpi=100)
        fig.patch.set_facecolor('#2B2B2B') # Match CustomTkinter dark theme
        ax.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=90, colors=colors, textprops={'color':"w"})
        ax.axis('equal')

        self.canvas = FigureCanvasTkAgg(fig, master=self.chart_frame)
        self.canvas.draw()
        self.canvas.get_tk_widget().pack(side="top", fill="both", expand=True)

    # ... (remove_and_recalibrate and reset_exclusions are the same)
    def remove_and_recalibrate(self, item_name):
        self.exclusion_list.append(item_name)
        self.status_label.configure(text=f"Excluding '{item_name}'. Recalibrating...")
        self.start_meal_search()
    
    def reset_exclusions(self):
        self.exclusion_list.clear()
        self.status_label.configure(text="Exclusion list cleared. Ready to search.")
        for widget in self.result_frame.winfo_children(): widget.destroy()

if __name__ == "__main__":
    def cleanup_old_caches():
        today_str = datetime.now().strftime('%Y-%m-%d')
        for filename in os.listdir('.'):
            if filename.startswith("menu_cache_") and today_str not in filename:
                try: os.remove(filename)
                except OSError: pass
    cleanup_old_caches()
    meal_finder_engine = MealFinder()
    app = App(meal_finder_engine)
    app.mainloop()

/var/folders/hf/x9ynj3l90c32gh_f1v1ccqw80000gn/T/ipykernel_90967/3473243160.py:314: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, ax = plt.subplots(figsize=(4, 2.5), dpi=100)
